In [1]:
from llama_parse import LlamaParse

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
#
from groq import Groq
from langchain_groq import ChatGroq
#
import joblib
import os
import nest_asyncio  # noqa: E402
nest_asyncio.apply()

In [2]:
llamaparse_api_key = "llx-RGNfIFYjgNsijUxFkoqVnjDQYRqCkYbh2tYV53LVJFl85BGz"
groq_api_key = "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"

In [3]:
def load_or_parse_data():
    data_file = "./data/parsed_data.pkl"

    if os.path.exists(data_file):
        # Load the parsed data from the file
        parsed_data = joblib.load(data_file)
    else:
        # Perform the parsing step and store the result in llama_parse_documents
        parsingInstructionUber10k = """Le document fourni est déposé par la BANQUE AL SALAM d'Algérie,

Il contient les conditions de la banque pour l'année 2024 et les coûts de chaque service.

Il contient de nombreuses tableaux.

Essayez d'être précis lors de la réponse aux questions."""
        parser = LlamaParse(api_key=llamaparse_api_key,
                            result_type="markdown",
                            parsing_instruction=parsingInstructionUber10k,
                            max_timeout=5000,)
        llama_parse_documents = parser.load_data("data/loi.pdf")


        # Save the parsed data to a file
        print("Saving the parse results in .pkl format ..........")
        joblib.dump(llama_parse_documents, data_file)

        # Set the parsed data to the variable
        parsed_data = llama_parse_documents

    return parsed_data

In [7]:
def create_vector_database():
    """
    Creates a vector database using document loaders and embeddings.

    This function loads urls,
    splits the loaded documents into chunks, transforms them into embeddings using OllamaEmbeddings,
    and finally persists the embeddings into a Chroma vector database.

    """
    # Call the function to either load or parse the data
    llama_parse_documents = load_or_parse_data()
    print(llama_parse_documents[0].text[:300])

    with open('data/output.md', 'a',encoding="utf-8") as f:  # Open the file in append mode ('a')
        for doc in llama_parse_documents:
            f.write(doc.text + '\n')

    markdown_path = "data/output.md"
    loader = UnstructuredMarkdownLoader(markdown_path)

   #loader = DirectoryLoader('data/', glob="**/*.md", show_progress=True)
    documents = loader.load()
    # Split loaded documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    #len(docs)
    print(f"length of documents loaded: {len(documents)}")
    print(f"total number of document chunks generated :{len(docs)}")
    #docs[0]

    # Initialize Embeddings
    embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

    # Create and persist a Chroma vector database from the chunked documents
    vs = Chroma.from_documents(
        documents=docs,
        embedding=embed_model,
        persist_directory="chroma_db_llamaparse1",  # Local mode with in-memory storage only
        collection_name="rag"
    )

    #query it
    #query = "what is the agend of Financial Statements for 2022 ?"
    #found_doc = qdrant.similarity_search(query, k=3)
    #print(found_doc[0][:100])
    #print(qdrant.get())

    print('Vector DB created successfully !')
    return vs,embed_model

In [11]:
vs,embed_model = create_vector_database()

#

# Conditions de la Banque Al Salam - 2024

# Conditions de la Banque Al Salam - 2024

Le document fourni est déposé par la BANQUE AL SALAM d'Algérie, il contient les conditions de la banque pour l'année 2024 et les coûts de chaque service.

# Tableau des Conditions de la Banque Al Salam - 2024

|
length of documents loaded: 1
total number of document chunks generated :89


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 5005.14it/s]


Vector DB created successfully !


In [20]:
chat_model = ChatGroq(temperature=0,
                      model_name="mixtral-8x7b-32768",
                      api_key=groq_api_key,)

vectorstore = Chroma(embedding_function=embed_model,
                      persist_directory="chroma_db_llamaparse1",
                      collection_name="rag")
retriever=vectorstore.as_retriever(search_kwargs={'k': 3})

In [25]:
custom_prompt_template = """Utilisez les éléments d'information suivants pour répondre à la question de l'utilisateur.
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

Contexte : {context}
Question : {question}

N'affichez que la réponse utile ci-dessous et rien d'autre.
Réponse utile :
"""

In [26]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [27]:
prompt = set_custom_prompt()
prompt

PromptTemplate(input_variables=['context', 'question'], template="Utilisez les éléments d'information suivants pour répondre à la question de l'utilisateur.\nSi vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.\n\nContexte : {context}\nQuestion : {question}\n\nN'affichez que la réponse utile ci-dessous et rien d'autre.\nRéponse utile :\n")

In [28]:
qa = RetrievalQA.from_chain_type(llm=chat_model,
                               chain_type="stuff",
                               retriever=retriever,
                               return_source_documents=True,
                               chain_type_kwargs={"prompt": prompt})

In [29]:
response = qa.invoke({"query": "Quel est le montant de la comission Virements intra agences ASBA"})

In [30]:
response['result']

'La commission pour un virement intra agences ASBA est gratuite.'

In [35]:
response = qa.invoke({"query": "Quel est le plafond hebdomadaire par type de carte CIB"})

In [36]:
response['result']

'Carte Classique : Plafond hebdomadaire de 50% du revenu\nCarte Premium : Plafond hebdomadaire de 50% à 80% du revenu\nCarte Gold : Plafond hebdomadaire de 100% du revenu'